This notebook is my attempt at pulling data from a library and prompting a user to input the information needed to calculate the location of a subducting slab.  I will be pulling heavily from the IntroToSlab2.0 python notebook

To run this notebook you will need to instal the following packages:

pip install git+git://github.com/usgs/libcomcat.git
pip install git+git://github.com/usgs/neicio.git

The cell below indicates to IPython that all plots should be rendered inline and not in a separate window.

In [ ]:
%matplotlib inline

Imports needed to run the code.

In [ ]:
#stdlib imports
from datetime import datetime
import urllib2
import os.path

#usgs imports
from slab2.slabutils import getEventsInCircle
from slab2.slabutils import getEventsInEllipse
from slab2.slabutils import heading
from libcomcat.comcat import getEventData,getTimeSegments
from neicio.gmt import GMTGrid

#third party imports
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy.io import netcdf
import math

If the user wishes to create a grid based on a slab1.0 image from the USGS website the we have the following

In [ ]:
recreate_grid = raw_input ('Do you wish to create a grid based on an area from slab1.0?\n>("y" for yes): ')
if recreate_grid == 'y':
    print """
    Available slab regions from slab1.0:
    
    Alaska-Aleutians         > alu_slab1
    Central America          > mex_slab1
    Cacadia                  > cas_slab1
    Izu-Bonin                > izu_slab1
    Kermadec-Tonga           > ker_slab1
    Kamchatka/Kurils/Japan   > kur_slab1
    Philippines              > phi_slab1
    Ryukyu                   > ryu_slab1
    Santa Cruz Islands       > van_slab1
    Scotia                   > sco_slab1
    Solomon Islands          > sol_slab1
    South America            > sam_slab1
    Sumatra-Java             > sum_slab1
    """
    # With the given choices above a region is selected with the input below
    selected_region = raw_input ("Enter the region you wish to study:\n>")
    # The url to pull from is determined
    website = 'http://earthquake.usgs.gov/data/slab/images/%s.jpg' % selected_region
    
    print 'You have chosen to analyze a slab in the %s region.' % selected_region 

In [ ]:
Image(url = website)

In [ ]:
if recreate_grid != 'y':
    print 'In order to confine the desired area please' \
    'enter the longitudinal and latitudinal range that is desired.\n'

    lonmin = input ('Enter the desired longitudinal Minimum: \n>')
    lonmax = input ('Enter the desired longitudinal Maximum: \n>')
    latmin = input ('Enter the desired latitudinal Minimum: \n>')
    latmax = input ('Enter the desired latitudinal Maximum: \n>')
  
    #create arrays of lat/lon values
    lons = np.arange(lonmin,lonmax,0.5)
    lats = np.arange(latmin,latmax,0.5)
    bounds = (lons.min(),lons.max(),lats.min(),lats.max())

In [ ]:
if recreate_grid == 'y':
    url = 'http://earthquake.usgs.gov/data/slab/models/%s.0_clip.grd' % selected_region
    fname = '%s.0_clip.grd' % selected_region
    if not os.path.isfile(fname):
        fh = urllib2.urlopen(url)
        data = fh.read()
        fh.close()
        f = open(fname,'wb')
        f.write(data)
        f.close()

    depgrid = GMTGrid(fname)
    xmin,xmax,ymin,ymax = depgrid.getRange()
    #create arrays of lat/lon values
    lons = np.arange(xmin,xmax,0.5)
    lats = np.arange(ymin,ymax,0.5)
    bounds = (lons.min(),lons.max(),lats.min(),lats.max())
    print bounds

In [ ]:
if recreate_grid == 'y':
    url = 'http://earthquake.usgs.gov/data/slab/models/%s.0_strclip.grd' % selected_region
    fname = '%s.0_strclip.grd' % selected_region
    if not os.path.isfile(fname):
        fh = urllib2.urlopen(url)
        data = fh.read()
        fh.close()
        f = open(fname,'wb')
        f.write(data)
        f.close()

    strgrid = GMTGrid(fname)

    url = 'http://earthquake.usgs.gov/data/slab/models/%s.0_dipclip.grd' % selected_region
    fname = '%s.0_dipclip.grd' % selected_region
    if not os.path.isfile(fname):
        fh = urllib2.urlopen(url)
        data = fh.read()
        fh.close()
        f = open(fname,'wb')
        f.write(data)
        f.close()

    dipgrid = GMTGrid(fname)

In [ ]:
print dipgrid
print selected_region

Regions that work:
sol
sum
mex
cas
ker
kur
phi
ryu
van
sco

Regions that don't work:
sam (?total size of new array must be unchanged?)
izu (?total size of new array must be unchanged?)
alu (?states that the file is not a valid NetCDF 3 file?)